In [11]:
%pip install --upgrade --quiet langchain-pinecone langchain-openai langchain pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [12]:

%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [42]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_docs/tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [52]:
document_list[52]

Document(metadata={'source': './tax_docs/tax_with_markdown.docx', 'text': '제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이

In [50]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [54]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents([], embedding, index_name=index_name)

batch_size = 100

for i in range(0, len(document_list), batch_size):
    batch = document_list[i: i+batch_size]
    database.add_documents(batch)

In [74]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# query = '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query)

In [67]:
retrieved_docs

[Document(id='476d142c-d6c5-42cf-a9e2-db7105642ebc', metadata={'source': './tax_docs/tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴

In [65]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [66]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\1\miniconda3\envs\disRagEnv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [68]:
from langchain.chains import RetrievalQA
retriever = database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
retriever.invoke(query)

[Document(id='46d52f4b-8dff-4f8d-b1c1-a50445f0ef99', metadata={'source': './tax_docs/tax_with_markdown.docx'}, page_content='2. 조정대상지역에 있는 주택으로서 1세대가 1주택과 조합원입주권 또는 분양권을 1개 보유한 경우의 해당 주택. 다만, 대통령령으로 정하는 장기임대주택 등은 제외한다.\n\n3. 조정대상지역에 있는 주택으로서 대통령령으로 정하는 1세대 3주택 이상에 해당하는 주택\n\n4. 조정대상지역에 있는 주택으로서 1세대가 주택과 조합원입주권 또는 분양권을 보유한 경우로서 그 수의 합이 3 이상인 경우 해당 주택. 다만, 대통령령으로 정하는 장기임대주택 등은 제외한다.\n\n⑧ 그 밖에 양도소득 산출세액의 계산에 필요한 사항은 대통령령으로 정한다.<개정 2014. 12. 23., 2016. 12. 20., 2017. 12. 19.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제104조(양도소득세의 세율) ① 거주자의 양도소득세는 해당 과세기간의 양도소득과세표준에 다음 각 호의 세율을 적용하여 계산한 금액(이하 “양도소득 산출세액”이라 한다)을 그 세액으로 한다. 이 경우 하나의 자산이 다음 각 호에 따른 세율 중 둘 이상에 해당할 때에는 해당 세율을 적용하여 계산한 양도소득 산출세액 중 큰 것을 그 세액으로 한다. <개정 2010. 12. 27., 2014. 1. 1., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2020. 8. 18., 2020. 12. 29., 2022. 12. 31.>\n\n1. 제94조제1항제1호ㆍ제2호 및 제4호에 따른 자산\n\n\u3000제55조제1항에 따른 세율(분양권의 경우에는 양도소득 과세표준의 100분의 60)\n\n2. 제94조제1항제1호 및 제2호에서 규정하는 자산으로서 그 보유기간이

In [69]:
ai_message = qa_chain({"query": query})

In [70]:
ai_message

{'query': '연봉 5천만원의 소득세를 구체적으로 계산하고, 세율 및 공제 내용을 포함해 설명해 주세요.',
 'result': '연봉 5천만원의 경우, 소득세는 과세표준 구간 1,400만원 초과 5,000만원 이하에 해당합니다. 따라서 세액은 기본 84만원에다가, 1,400만원을 초과하는 금액인 3,600만원(5,000만원 - 1,400만원)의 15%를 더하게 됩니다. 계산하면 총 소득세는 84만원 + 540만원(3,600만원 x 15%) = 624만원입니다.'}

In [81]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전 : {dictionary}

    질문 : {{question}}                           
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [76]:
new_question = dictionary_chain.invoke({"question": query})

In [77]:
new_question

'질문 : 연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [78]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [79]:
ai_response = tax_chain.invoke({"question": query})

In [80]:
ai_response

{'query': '거주자 연봉 5천만원의 소득세는 얼마인가요?',
 'result': '거주자의 연봉 5,000만원에 대한 소득세는 624만원입니다. 이 금액은 종합소득 과세표준에서 5,000만원 이하의 구간에 해당하며, 해당 구간의 계산식은 "84만원 + (1,400만원을 초과하는 금액의 15퍼센트)"를 적용한 결과입니다.'}